In [30]:
# import os
# import torch
# from IPython.display import clear_output

# # Clear CUDA memory and cache
# torch.cuda.empty_cache()
# clear_output()

# # Verify memory is cleared
# print(f"CUDA Memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f}GB (should be 0.00GB)")
# print(f"CUDA Memory reserved: {torch.cuda.memory_reserved()/1024**3:.2f}GB (should be 0.00GB)")

# # Environment setup
# os.environ['NO_ALBUMENTATIONS_UPDATE'] = '1'  # Disable Albumentations update warnings
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'  # Prevent fragmentation
# print("Done")

# Model - 1 --> working : True

In [32]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import timm
from scipy.stats import zscore
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.preprocessing import RobustScaler

# ---------------------- 1. Configuration ----------------------
SEED = 42
BATCH_SIZE =20
EPOCHS = 43
EARLY_STOPPING_PATIENCE = 11
torch.manual_seed(SEED)
np.random.seed(SEED)

# ---------------------- 2. Enhanced Data Preparation ----------------------
def load_data():
    train_df = pd.read_csv('/kaggle/input/labels-geo-data-smai/labels_train.csv')
    val_df = pd.read_csv('/kaggle/input/labels-geo-data-smai/labels_val.csv')
    
    train_df['filepath'] = train_df['filename'].apply(lambda x: os.path.join('/kaggle/input/geo-data-final-smai/final_datasets/images_train', x))
    val_df['filepath'] = val_df['filename'].apply(lambda x: os.path.join('/kaggle/input/geo-data-final-smai/final_datasets/images_val', x))
    
    def remove_outliers(df):
        for col in ['latitude', 'longitude']:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 3 * IQR
            upper_bound = Q3 + 3 * IQR
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
        return df
    
    train_df = remove_outliers(train_df)
    val_df = remove_outliers(val_df)
    return train_df, val_df

train_df, val_df = load_data()

# ---------------------- 3. Advanced Data Augmentation & Normalization ----------------------
class GeoDataset(Dataset):
    def __init__(self, df, transform=None, is_train=False):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.is_train = is_train
        
        if 'latitude' in df.columns and 'longitude' in df.columns:
            self.scaler_lat = RobustScaler()
            self.scaler_lon = RobustScaler()
            self.scaler_lat.fit(df[['latitude']].values.reshape(-1, 1))
            self.scaler_lon.fit(df[['longitude']].values.reshape(-1, 1))

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['filepath']).convert('RGB')
        image = np.array(image)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        if 'latitude' in row and 'longitude' in row:
            latitude = self.scaler_lat.transform([[row['latitude']]])[0][0]
            longitude = self.scaler_lon.transform([[row['longitude']]])[0][0]
            return image, torch.tensor([latitude, longitude], dtype=torch.float32)
        else:
            return image, torch.tensor([0, 0], dtype=torch.float32)

train_transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Transpose(p=0.5),
    A.Affine(
        translate_percent=(-0.1, 0.1),
        scale=(0.85, 1.15),
        rotate=(-15, 15),
        p=0.5
    ),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.CoarseDropout(
        max_holes=8,
        min_holes=1,
        max_height=32,
        min_height=8,
        max_width=32,
        min_width=8,
        fill_value=0,
        mask_fill_value=None,
        p=0.5
    ),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

train_ds = GeoDataset(train_df, train_transform, is_train=True)
val_ds = GeoDataset(val_df, val_transform)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

# ---------------------- 4. Enhanced Model Architecture ----------------------
class GeoLocalizationModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model('tf_efficientnetv2_m', pretrained=True, num_classes=0)
        
        self.attention = nn.Sequential(
            nn.Linear(1280, 512),
            nn.SiLU(),
            nn.Linear(512, 1280),
            nn.Sigmoid())
        
        self.coord_head = nn.Sequential(
            nn.Linear(1280, 1024),
            nn.SiLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.SiLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 2))
        
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        features = self.backbone(x)
        attn_weights = self.attention(features)
        weighted_features = features * attn_weights
        coords = self.coord_head(weighted_features)
        return coords

model = GeoLocalizationModel().cuda()

# ---------------------- 5. Enhanced Training Setup ----------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3, weight_decay=1e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
criterion = nn.MSELoss()
scaler = torch.cuda.amp.GradScaler()

# ---------------------- 6. Enhanced Training Loop ----------------------
def evaluate(model, dataloader, denormalize=True):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.cuda()
            output = model(images)
            preds.append(output.cpu().numpy())
            targets.append(labels.cpu().numpy())
    
    preds = np.vstack(preds)
    targets = np.vstack(targets)
    
    if denormalize and hasattr(dataloader.dataset, 'scaler_lat'):
        preds_lat = dataloader.dataset.scaler_lat.inverse_transform(preds[:, 0].reshape(-1, 1))
        preds_lon = dataloader.dataset.scaler_lon.inverse_transform(preds[:, 1].reshape(-1, 1))
        targets_lat = dataloader.dataset.scaler_lat.inverse_transform(targets[:, 0].reshape(-1, 1))
        targets_lon = dataloader.dataset.scaler_lon.inverse_transform(targets[:, 1].reshape(-1, 1))
        
        preds = np.hstack([preds_lat, preds_lon])
        targets = np.hstack([targets_lat, targets_lon])
    
    mse_lat = mean_squared_error(targets[:, 0], preds[:, 0])
    mse_lon = mean_squared_error(targets[:, 1], preds[:, 1])
    avg_mse = 0.5 * (mse_lat + mse_lon)
    
    return avg_mse, mse_lat, mse_lon, preds, targets

best_val_mse = float('inf')
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    
    for images, labels in tqdm(train_dl, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images, labels = images.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        
        with autocast():
            output = model(images)
            loss = criterion(output, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item()
    
    scheduler.step()
    
    val_avg_mse, val_mse_lat, val_mse_lon, _, _ = evaluate(model, val_dl)
    
    print(f"\nEpoch {epoch+1}:")
    print(f"Train Loss: {running_loss/len(train_dl):.4f}")
    print(f"Val Avg MSE: {val_avg_mse:.2f} = (Lat MSE: {val_mse_lat:.2f} + Lon MSE: {val_mse_lon:.2f})")
    
    if val_avg_mse < best_val_mse:
        best_val_mse = val_avg_mse
        patience_counter = 0
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_mse': val_avg_mse,
            'epoch': epoch,
        }, 'best_model_ll_k.pth')
        print("Saved new best model")
    else:
        patience_counter += 1
        if patience_counter >= EARLY_STOPPING_PATIENCE:
            print(f"Early stopping after {epoch+1} epochs")
            break

# ---------------------- 7. Load Best Model ----------------------
checkpoint = torch.load('best_model_ll_k.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"\nLoaded best model from epoch {checkpoint['epoch']+1} with Val MSE: {checkpoint['val_mse']:.2f}")

# ---------------------- 8. Prepare Test Data ----------------------
test_images = sorted(os.listdir('/kaggle/input/geo-data-final-smai/final_datasets/images_test'))
test_df = pd.DataFrame({
    'filename': test_images,
    'filepath': [os.path.join('/kaggle/input/geo-data-final-smai/final_datasets/images_test', img) for img in test_images]
})

test_ds = GeoDataset(test_df, val_transform)
test_ds.scaler_lat = train_ds.scaler_lat
test_ds.scaler_lon = train_ds.scaler_lon
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# ---------------------- 9. Generate Predictions ----------------------
# Predict on validation set
_, _, _, val_preds, val_targets = evaluate(model, val_dl)

# Predict on test set
model.eval()
test_preds = []
with torch.no_grad():
    for images, _ in test_dl:
        images = images.cuda()
        output = model(images)
        output = output.cpu().numpy()
        output_lat = test_ds.scaler_lat.inverse_transform(output[:, 0].reshape(-1, 1))
        output_lon = test_ds.scaler_lon.inverse_transform(output[:, 1].reshape(-1, 1))
        test_preds.append(np.hstack([output_lat, output_lon]))

test_preds = np.vstack(test_preds)



/tmp/ipykernel_9410/1034464240.py:98: UserWarning: Argument(s) 'max_holes, min_holes, max_height, min_height, max_width, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(
Epoch 1/43: 100%|██████████| 323/323 [01:21<00:00,  3.98it/s]



Epoch 1:
Train Loss: 2.5584
Val Avg MSE: 941872.56 = (Lat MSE: 808172.75 + Lon MSE: 1075572.38)
Saved new best model


Epoch 2/43: 100%|██████████| 323/323 [01:20<00:00,  3.99it/s]



Epoch 2:
Train Loss: 0.4413
Val Avg MSE: 913621.44 = (Lat MSE: 789858.50 + Lon MSE: 1037384.38)
Saved new best model


Epoch 3/43: 100%|██████████| 323/323 [01:20<00:00,  4.03it/s]


...


Epoch 41:
Train Loss: 0.0544
Val Avg MSE: 128694.84 = (Lat MSE: 120648.10 + Lon MSE: 136741.59)
Saved new best model


Epoch 42/43: 100%|██████████| 323/323 [01:20<00:00,  4.01it/s]



Epoch 42:
Train Loss: 0.0509
Val Avg MSE: 124341.77 = (Lat MSE: 114479.22 + Lon MSE: 134204.33)
Saved new best model


Epoch 43/43: 100%|██████████| 323/323 [01:21<00:00,  3.98it/s]



Epoch 43:
Train Loss: 0.0503
Val Avg MSE: 129210.16 = (Lat MSE: 119418.03 + Lon MSE: 139002.30)

Loaded best model from epoch 42 with Val MSE: 124341.77


In [36]:
import numpy as np
import pandas as pd

# anomaly_ids to skip when assigning IDs
anomaly_ids = {95, 145, 146, 158, 159, 160, 161}

# Make sure you have 732 predictions
# all_preds should be of shape (732, 2)
all_preds = np.vstack([val_preds, test_preds])  # should be (732, 2)

submission_data = []
pred_idx = 0  # index for predictions

# Loop over IDs from 0 to 737 (inclusive)
for new_id in range(738):
    if new_id in anomaly_ids:
        continue  # Skip assigning prediction to this ID

    # Assign prediction to current non-anomalous ID
    submission_data.append({
        'id': new_id,
        'Latitude': int(round(all_preds[pred_idx][0])),
        'Longitude': int(round(all_preds[pred_idx][1]))
    })
    pred_idx += 1  # Move to next prediction

# Final DataFrame
submission = pd.DataFrame(submission_data)

# Sanity checks
print(f"Total IDs: 738")
print(f"Anomalies skipped: {len(anomaly_ids)}")
print(f"Predictions used: {pred_idx}")
print(f"Submission rows: {len(submission)}")

# Check for duplicate IDs
if len(submission['id']) != len(submission['id'].unique()):
    print("\nWarning: Duplicate IDs found")
else:
    print("\nNo duplicate IDs found")

# Save to CSV
submission.to_csv('2022101002_31_Lat_Long_k6_ff.csv', index=False)
print("\nFinal submission sample (around skipped ID 95):")
print(submission[submission['id'].between(90, 100)])


Total IDs: 738
Anomalies skipped: 7
Predictions used: 731
Submission rows: 731

No duplicate IDs found

Final submission sample (around skipped ID 95):
     id  Latitude  Longitude
90   90    220335     143744
91   91    220273     143980
92   92    220462     143877
93   93    219829     142008
94   94    220016     142151
95   96    219183     141141
96   97    219335     141383
97   98    219863     141715
98   99    218660     143572
99  100    221226     142910
